In [21]:
include("../LiPoSID.jl")

using LinearAlgebra

using DynamicPolynomials

using SymPy

#using Plots

using Dates

using HDF5

data_dir = "../DATA/"

γ_list = [ "0.079477",  "0.25133", "0.79477", "2.5133", "7.9477", "25.133",  "79.477", "251.33"]

γ_list_disp = [ "0.0795",  "0.2513", "0.7947", "2.5133", "7.9477", "25.133",  "79.477", "251.33"];

b⁰ = [0, 0, 1, 1]
b¹ = [0, 0, -1, 1]
bˣ = [1, 0, 0, 1]
bʸ = [1, 1, 0, 1]


function ⊗(A::AbstractMatrix, B::AbstractMatrix)
    return kron(A,B)
end

function LME_operator_symbolic(H, J)
    
    Id = I(2)   
    U = im*(Id ⊗ H - transpose(H) ⊗ Id) #was - "minus"
    D = sum( 2*transpose(j')⊗j-Id⊗(j'*j) - transpose(j)*transpose(j')⊗Id for j in J )/2 
    
    Lᴸᴹᴱ = U + D
    
    return Lᴸᴹᴱ
end

function SumCoeffs2(poly, s)
    p = SymPy.expand(poly)
    rest_poly = SymPy.subs(p, s => 0)
    degrees = range(1,convert(Int32,SymPy.degree(p, s)))
    
    obj = sum(p.coeff(s^i)^2 for i in degrees) + rest_poly^2  
    
    return obj
end 

function sympy_to_dynamicpoly(sympy_expr::Sym) #
    # Extract variables from the SymPy expression
    vars = free_symbols(sympy_expr)

    # Convert SymPy variables to string and then to Symbol
    var_symbols = [Symbol(string(v)) for v in vars]

    # Create DynamicPolynomials variables using @polyvar macro
    @eval begin
        @polyvar $(var_symbols...)
    end

    # Map SymPy variables to DynamicPolynomials variables
    var_map = Dict(vars[i] => eval(var_symbols[i]) for i in 1:length(vars))

    # Substitute using subs and convert to DynamicPolynomial
    dynamic_poly_expr = SymPy.subs(sympy_expr, var_map)
    dynamic_poly = eval(Meta.parse(string(dynamic_poly_expr)))
    return dynamic_poly
end


function TF_objective(Aˢʸᵐᵇ, Aˢⁱᵈ, b)

    @syms s

    Gˢʸᵐᵇ = inv(I(4)*s - Aˢʸᵐᵇ)*b

    Gˢⁱᵈ = inv(I(4)*s - Aˢⁱᵈ)*b

    #ΔG = SymPy.together.(Gˢʸᵐᵇ[1:3] - Gˢⁱᵈ[1:3])
    ΔG = SymPy.together.(Gˢʸᵐᵇ - Gˢⁱᵈ)

    polys = SymPy.numerator.(ΔG)

    obj = sum(SumCoeffs2.(polys.^2, s))

    #objective = sympy_to_dynamicpoly(obj)

    return obj

end

function read_Ac(models_file_name, γ)
    h5open(models_file_name, "r") do file
        Ac = read(file["gamma_"*string(γ)]["Ac"])
        return Ac
    end
end

function dmd_Ac(γ)

    data_dir = "../DATA/"
    
    ρᵉ, tᵉ = LiPoSID.get_rho_series(data_dir*"State_B1_2CUT_data.h5", γ)
    ρᵍ, tᵍ = LiPoSID.get_rho_series(data_dir*"State_B2_2CUT_data.h5", γ)
    ρˣ, tˣ = LiPoSID.get_rho_series(data_dir*"State_B3_2CUT_data.h5", γ)
    ρʸ, tʸ = LiPoSID.get_rho_series(data_dir*"State_B4_2CUT_data.h5", γ)

    Δt = tᵉ[2]-tᵉ[1]

    lᵉ = length(ρᵉ); lᵍ = length(ρᵍ); lˣ = length(ρˣ); lʸ = length(ρʸ)
    lᵐᵃˣ = min(lᵉ, lᵍ,  lˣ, lʸ)  #choose time limit by shortest series

    Aᴰᴹᴰ⁻ˢᴮ = LiPoSID.direct_DMD_01XY_b4_A([ρᵉ[1:lᵐᵃˣ], ρᵍ[1:lᵐᵃˣ], ρˣ[1:lᵐᵃˣ], ρʸ[1:lᵐᵃˣ]])
    Acᴰᴹᴰ⁻ˢᴮ = log(Aᴰᴹᴰ⁻ˢᴮ)/Δt

    return Acᴰᴹᴰ⁻ˢᴮ

end

function sb_TF_objective(γ, Acˢʸᵐᵇ)
    models_file_name="../MODELS/DMD_b4_LTI_trn4_2023-Nov-29_at_17-38.h5"
    Acᴰᴹᴰ = read_Ac(models_file_name, γ)
    obj =  TF_objective(Acˢʸᵐᵇ, Acᴰᴹᴰ, bˣ) + TF_objective(Acˢʸᵐᵇ, Acᴰᴹᴰ, bʸ) + TF_objective(Acˢʸᵐᵇ, Acᴰᴹᴰ, b¹) + TF_objective(Acˢʸᵐᵇ, Acᴰᴹᴰ, b⁰)
    objective = sympy_to_dynamicpoly(obj)
    return objective
end



sb_TF_objective (generic function with 1 method)

In [22]:
w = SymPy.symbols("w", real=true)
a = SymPy.symbols("a", real=true)

Hᴸᴹᴱ = [ w        0
         0        0   ]


Jᴸᴹᴱ = [ 0     a
         0     0. + 0im  ]

# a = √γ

Lᴸᴹᴱ = LME_operator_symbolic(Hᴸᴹᴱ, [Jᴸᴹᴱ])

M =   [ 0  0    1  1
        1  -im  0  0 
        1  im   0  0
        0  0   -1  1 ]/2

Aᴸᴹᴱ = real.(inv(M) * Lᴸᴹᴱ * M)

4×4 Matrix{Sym{PyCall.PyObject}}:
 -0.5*a^2    -1.0⋅w         0        0
    1.0⋅w  -0.5*a^2         0        0
        0         0  -1.0*a^2  1.0*a^2
        0         0         0        0

In [23]:
date_and_time_string =  string(Dates.format(now(), "yyyy-u-dd_at_HH-MM"))
objectives_file = "TS_POP_objectives_simple_" * date_and_time_string * ".h5"

models_dir = "../MODELS/"

γ_list = [ "0.079477",  "0.25133", "0.79477", "2.5133", "7.9477", "25.133", "79.477", "251.33"]

for γᵢ in  γ_list

    println(γᵢ)

    dmd_ts_pop_objective = sb_TF_objective(γᵢ, Aᴸᴹᴱ)
    println(dmd_ts_pop_objective)

    h5open(models_dir*objectives_file,"cw") do fid  # read-write, create file if not existing, preserve existing contents

        γ_group = create_group(fid, "gamma_"*γᵢ)
        γ_group["dmd_ts_pop_objective"] = string(dmd_ts_pop_objective)
                        
    end 

end


println(models_dir*objectives_file)

0.079477


3.715288226045447e67 - 2.5143356191413642e67*w + 7.670890506656457e67*w^2 + 2.6429643081656913e67*a^2 - 2.80158673369877e67*w^3 - 7.868010711432142e67*a^2*w + 2.059465491132775e67*w^4 + 4.201866438181009e67*a^2*w^2 + 3.916467595203904e67*a^4 - 2.9196787817521443e66*w^5 - 3.989159885780594e67*a^2*w^3 - 2.4023805620386413e67*a^4*w + 1.6769931913794347e65*w^6 + 4.3968237695143565e66*a^2*w^4 + 2.9971182033681085e67*a^4*w^2 + 7.754896377531028e66*a^6 - 4.3615708348570925e63*w^7 - 1.700447426675542e65*a^2*w^5 - 2.9795607701735195e66*a^4*w^3 - 1.0520915263075952e67*a^6*w + 4.2839549627051876e61*w^8 + 2.2203132689706346e63*a^2*w^6 + 1.2767423246003138e65*a^4*w^4 + 1.4888676738319833e66*a^6*w^2 + 2.5969281594140883e66*a^8 - 3.2711781261428166e63*a^4*w^5 - 8.502237133377711e64*a^6*w^3 - 5.624102686838706e65*a^8*w + 4.283954962705176e61*a^4*w^6 + 1.665234951727973e63*a^6*w^4 + 3.2393493891651244e64*a^8*w^2 + 9.741543286334798e64*a^10 - 8.1779453153570425e62*a^8*w^3 - 1.0627796416722147e64*a^10*w 

7.655322260029137e67 - 1.0432028877693255e68*w + 1.7274497824054477e68*w^2 + 1.0581912818852446e68*a^2 - 1.1004252031378692e68*w^3 - 1.8077619255605908e68*a^2*w + 4.752055972197257e67*w^4 + 1.6504464757638102e68*a^2*w^2 + 8.937931480441782e67*a^4 - 5.976044265828763e66*w^5 - 8.829852223965582e67*a^2*w^3 - 8.921408110680701e67*a^4*w + 3.2501648382095913e65*w^6 + 8.992454118145482e66*a^2*w^4 + 6.626424509697454e67*a^4*w^2 + 2.906392353924719e67*a^6 - 8.176234156018012e63*w^7 - 3.356517052497513e65*a^2*w^5 - 6.208809884927013e66*a^4*w^3 - 2.4180938763631242e67*a^6*w + 7.841457977338361e61*w^8 + 4.284787007158858e63*a^2*w^6 + 2.521380196009596e65*a^4*w^4 + 3.087969327947989e66*a^6*w^2 + 5.843154301457848e66*a^8 - 6.132175617013503e63*a^4*w^5 - 1.6782585262487544e65*a^6*w^3 - 1.1786997046174547e66*a^8*w + 7.841457977338355e61*a^4*w^6 + 3.2135902553691412e63*a^6*w^4 + 6.512841908405001e64*a^8*w^2 + 2.0996394960290518e65*a^10 - 1.5330439042533746e63*a^8*w^3 - 2.0978231578109412e64*a^10*w + 2.

1.159041015194136e69 - 1.90636278539185e69*w + 2.83820726137799e69*w^2 + 1.9083976805272316e69*a^2 - 1.9903529088851455e69*w^3 - 2.9811190468294634e69*a^2*w + 7.471962923597161e68*w^4 + 2.982699202402866e69*a^2*w^2 + 1.4666893337025795e69*a^4 - 8.832248654960409e67*w^5 - 1.3705883259263458e69*a^2*w^3 - 1.5992451813875966e69*a^4*w + 4.601770972658825e66*w^6 + 1.3280745523775971e68*a^2*w^4 + 1.0265373192333295e69*a^4*w^2 + 5.186833943307905e68*a^6 - 1.1168203477635957e65*w^7 - 4.780344243212746e66*a^2*w^5 - 9.223919356284783e67*a^4*w^3 - 3.7931774659489206e68*a^6*w + 1.0367609614013063e63*w^8 + 5.909436275029191e64*a^2*w^6 + 3.595349491621943e66*a^4*w^4 + 4.55859221883756e67*a^6*w^2 + 9.058917849196761e67*a^8 - 8.376152608226979e64*a^4*w^5 - 2.390172121606375e66*a^6*w^3 - 1.7539642981361708e67*a^8*w + 1.036760961401308e63*a^4*w^6 + 4.4320772062718984e64*a^6*w^4 + 9.348426884374415e65*a^8*w^2 + 3.0960145947339135e66*a^10 - 2.0940381520567464e64*a^8*w^3 - 2.9877151520079697e65*a^10*w + 3.8

3.9775354991397586e71 - 6.164307272697277e71*w + 9.420435767389391e71*w^2 + 6.151009756739628e71*a^2 - 6.428402453692123e71*w^3 - 9.863530882913545e71*a^2*w + 2.4923092748285877e71*w^4 + 9.608373506491208e71*a^2*w^2 + 4.823179300036851e71*a^4 - 2.774127029375471e70*w^5 - 4.618599980832878e71*a^2*w^3 - 5.145903029798791e71*a^4*w + 1.336159254344464e69*w^6 + 4.173801952034808e70*a^2*w^4 + 3.4482667953422774e71*a^4*w^2 + 1.6520910628709802e71*a^6 - 2.979430895765625e67*w^7 - 1.3808273589407713e69*a^2*w^5 - 2.8815136612772895e70*a^4*w^3 - 1.2636407969782138e71*a^6*w + 2.5354689339232276e65*w^8 + 1.5580618592064215e67*a^2*w^6 + 1.0442370396738179e69*a^4*w^4 + 1.3971337824958324e70*a^6*w^2 + 2.977613927201025e70*a^8 - 2.23457317182422e67*a^4*w^5 - 6.90413679470386e68*a^6*w^3 - 5.469954759833557e69*a^8*w + 2.5354689339232323e65*a^4*w^6 + 1.1685463944048183e67*a^6*w^4 + 2.715886596473223e68*a^8*w^2 + 8.842082362178287e68*a^10 - 5.58643292956055e66*a^8*w^3 - 8.630170993379823e67*a^10*w + 9.5080

8.520146563311732e75 - 9.050458013270769e75*w + 1.8470404576508888e76*w^2 + 8.81100529712671e75*a^2 - 9.441269398411841e75*w^3 - 1.8936440666112676e76*a^2*w + 4.775152106697265e75*w^4 + 1.3909006454653856e76*a^2*w^2 + 9.12417557494663e75*a^4 - 3.8856966803631865e74*w^5 - 9.209286007049924e75*a^2*w^3 - 7.481776487702309e75*a^4*w + 1.2823018258873874e73*w^6 + 5.859491877813511e74*a^2*w^4 + 6.811136674061256e75*a^4*w^2 + 2.2638164212860417e75*a^6 - 1.9462630756503017e71*w^7 - 1.2973083325629222e73*a^2*w^5 - 3.934677734092327e74*a^4*w^3 - 2.3930684099141026e75*a^6*w + 1.3256690612710667e69*w^8 + 7.238945483766198e70*a^2*w^6 + 1.0992664765488207e73*a^4*w^4 + 1.6985064078693484e74*a^6*w^2 + 5.489293452944668e74*a^8 - 1.4596973067377277e71*a^4*w^5 - 6.486541662814619e72*a^6*w^3 - 7.408133910003834e73*a^8*w + 1.325669061271067e69*a^4*w^6 + 5.4292091128246485e70*a^6*w^4 + 3.092016459205253e72*a^8*w^2 + 5.840835960399199e72*a^10 - 3.6492432668443217e70*a^8*w^3 - 8.108177078518284e71*a^10*w + 4.9

5.550930706543035e27 + 2.0370605008750335e28*w + 3.031815002636738e28*w^2 + 9.619184445639214e27*a^2 + 1.9617148442481026e28*w^3 + 2.9148493391557282e28*a^2*w + 1.0264284248371056e28*w^4 + 2.9701716441517938e28*a^2*w^2 + 6.781230351137353e27*a^4 - 4.658321684926985e26*w^5 + 9.913754550272549e27*a^2*w^3 + 1.40163737120123e28*a^4*w + 9.213211617488157e25*w^6 - 8.550121780331694e26*a^2*w^4 + 7.484669852064333e27*a^4*w^2 + 2.0981378685459323e27*a^6 + 3.9558877424826133e24*w^7 + 1.592943488838391e25*a^2*w^5 - 5.608104416425527e26*a^4*w^3 + 2.2173861045685426e27*a^6*w + 8.697376761339537e25*w^8 + 1.7643073084359855e26*a^2*w^6 + 2.0841912955898942e26*a^4*w^4 - 2.053750992073758e26*a^6*w^2 + 2.3715154644538694e26*a^8 + 2.966915806861963e24*a^4*w^5 + 7.964717444191955e24*a^6*w^3 - 1.1108809987984455e26*a^8*w + 8.697376761339535e25*a^4*w^6 + 1.3232304813269891e26*a^6*w^4 + 8.693479299670435e25*a^8*w^2 + 2.0944863252291648e24*a^10 + 7.417289517154911e23*a^8*w^3 + 9.95589680523995e23*a^10*w + 3.26

6.423908666624216e28 - 1.5756251481456121e29*w + 2.705906581515902e29*w^2 - 8.55882900322273e28*a^2 - 1.6401139428346637e29*w^3 + 2.7608088455707924e29*a^2*w + 1.31320905947716e29*w^4 - 2.476917660634918e29*a^2*w^2 + 7.342520647255602e28*a^4 - 8.156032530516298e27*w^5 + 1.3715501274377733e29*a^2*w^3 - 1.3034573336966222e29*a^4*w + 7.529680292651737e26*w^6 - 8.372277081301854e27*a^2*w^4 + 1.0186369649537119e29*a^4*w^2 - 2.399476012938041e28*a^6 - 1.699995290293348e26*w^7 + 1.6961644882990666e25*a^2*w^5 - 6.823057840121706e27*a^4*w^3 + 3.6225210504416197e28*a^6*w + 8.342800815912351e26*w^8 - 4.593142921051601e26*a^2*w^6 + 9.870834349654956e26*a^4*w^4 - 2.899380091600446e27*a^6*w^2 + 5.114526206206596e27*a^8 - 1.2749964677200101e26*a^4*w^5 + 8.480822441495423e24*a^6*w^3 - 1.1960124268731597e27*a^8*w + 8.34280081591235e26*a^4*w^6 - 3.444857190788701e26*a^6*w^4 + 3.523602119955279e26*a^8*w^2 - 2.0157770531874613e26*a^10 - 3.187491169300027e25*a^8*w^3 + 1.0601028051869274e24*a^10*w + 3.12855

1.6489274869824019e30 + 4.8408738653288277e30*w + 7.7975310637821e30*w^2 + 2.2811580623668643e30*a^2 + 4.6948554049114897e30*w^3 + 7.583611687227376e30*a^2*w + 3.1997803611732003e30*w^4 + 7.05416225493717e30*a^2*w^2 + 1.7979234791967132e30*a^4 - 1.498569407421589e29*w^5 + 3.0867635632024414e30*a^2*w^3 + 3.36239774605862e30*a^4*w + 4.960478666572243e27*w^6 - 2.094193876793781e29*a^2*w^4 + 2.3237304968454333e30*a^4*w^2 + 5.0885808823821944e29*a^6 + 9.859883592741615e25*w^7 + 4.3690411485748945e27*a^2*w^5 - 1.3885172588903425e29*a^4*w^3 + 7.243091949429228e29*a^6*w + 4.508315458556191e26*w^8 + 4.3901566130900005e26*a^2*w^6 + 3.9047860289703377e27*a^4*w^4 - 6.815927110436583e28*a^6*w^2 + 7.928856616372949e28*a^8 + 7.394912694556196e25*a^4*w^5 + 2.1845205742874478e27*a^6*w^3 - 2.5346872675873645e28*a^8*w + 4.50831545855619e26*a^4*w^6 + 3.292617459817504e26*a^6*w^4 + 1.0223032645028735e27*a^8*w^2 - 3.951106046130324e27*a^10 + 1.8487281736390474e25*a^8*w^3 + 2.7306507178593128e26*a^10*w + 1.6